In [1]:
from pathlib import Path

file = Path('data/discontinued/Sub_109_Static_R_025_035.000008.obj')
is_plot = True

## Load landmarks labelling

In [2]:
import pandas as pd

df = pd.read_pickle(file.with_suffix('.pkl'))
df

coord,x,y,z
landmark,,,
P1,-14.333711,-156.724152,125.381439
P10,24.738054,-137.472134,-114.750564
P11,42.515607,-89.507008,-56.807751
P12,-22.967116,-83.423289,-65.599388
P2,30.239462,-155.406915,118.342365
P3,-64.400307,-150.323647,71.673798
P4,39.423142,-145.082466,72.643708
P5,-60.787144,-146.948391,40.668815
P6,10.064317,-100.603288,26.267208


## Local frame

In [3]:
import pyvista as pv
from measure import frame
from mesh4d.analyse import crave

mesh = crave.fix_pvmesh_disconnect(pv.read(file), df.values)

# foot bottom cropped for estimating foot local frame
if is_plot:
    frame.plantar_clip(mesh, df).plot()

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd531519100_0&reconnect=auto' style='widt…

In [4]:
from measure import visual

axes_frame, origin = frame.estimate_foot_frame(mesh, df)

if is_plot:
    visual.plot_axes(origin, axes_frame, mesh)

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd531519820_1&reconnect=auto' style='widt…

In [5]:
mesh_clip = frame.foot_clip(mesh, df)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)

if is_plot:
    mesh_local.plot()

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd5313da7f0_2&reconnect=auto' style='widt…

In [6]:
df_local = frame.df2local(df, axes_frame, origin)
df_local

coord,x,y,z
landmark,,,
P1,111.621997,-0.054710,7.265864
P10,-131.629623,18.907197,12.718908
P11,-78.175344,41.508964,63.766608
P12,-81.827123,-24.470748,69.736165
P2,100.841012,43.779038,7.917992
P3,61.983233,-54.412097,10.927997
P4,54.026023,49.138004,15.595262
P5,30.643988,-53.389351,12.528702
P6,7.803199,16.074982,57.564384


## Metrics

In [7]:
results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

In [8]:
from measure import metric

results.append(
    {
        'file': str(file),
        'FL': metric.fl(df_local),
        'MBL': metric.mbl(df_local),
        'LBL': metric.lbl(df_local),
        'ABW': metric.abw(df_local),
        'OBW': metric.obw(df_local),
        'OHW': metric.ohw(df_local),
        'BH': metric.bh(df_local),
        'IH': metric.ih(df_local),
        'BA': metric.ba(df_local),
        'T1A': metric.t1a(df_local),
        'T5A': metric.t5a(df_local),
        'ABG': metric.abg(df_local, mesh_local),
        'IG': metric.ig(df_local, mesh_local),
    }
)

In [9]:
df_metric = pd.DataFrame(results).set_index('file')
df_metric

,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
data/discontinued/Sub_109_Static_R_025_035.000008.obj,243.25162,185.655646,162.273611,105.204477,103.550101,62.849027,57.564384,69.895626,73.696817,15.494203,11.179462,253.641738,249.545591


### Visualization

In [18]:
if is_plot:
    settings = {
        'FL': ['P1', 'P10', 'x'],
        'MBL': ['P4', 'P10', 'x'],
        'LBL': ['P5', 'P10', 'x'],
        'OBW': ['P4', 'P3', 'y'],
        'OHW': ['P9', 'P8', 'y'],
    }

    for name, [landmark1, landmark2, axis] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_dist_along_axis(scene, df_local, landmark1, landmark2, axis, name=name)
        scene.show()
        scene.screenshot(f'output/visual/{name}.png')

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd5422525b0_22&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd551d17fd0_23&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd5421cf220_24&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd511f59670_25&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd4f0e57700_26&reconnect=auto' style='wid…

In [19]:
if is_plot:
    settings = {
        'BH': 'P6',
        'IH': 'P7',
    }

    for name, landmark in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_height(scene, df_local, landmark, name=name)
        scene.show()
        scene.screenshot(f'output/visual/{name}.png')

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd4e05b86d0_27&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd4e05b8f10_28&reconnect=auto' style='wid…

In [28]:
if is_plot:
    settings = {
        'BA': ['P4', 'P5', 'P8'],
        'T1A': ['P4', 'P2', 'P8'],
        'T5A': ['P5', 'P3', 'P9'],
    }

    for name, [landmark_origin, landmark1, landmark2] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_angle(scene, df_local, landmark_origin, landmark1, landmark2, actue_angle=True, name=name)
        scene.show()
        scene.screenshot(f'output/visual/{name}.png')

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd553d84be0_45&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd4f0fb3310_46&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd5549d4a30_47&reconnect=auto' style='wid…

In [29]:
if is_plot:
    settings = {
        'ABW': ['P4', 'P5'],
    }

    for name, [landmark1, landmark2] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_dist(scene, df_local, landmark1, landmark2, name=name)
        scene.show()
        scene.screenshot(f'output/visual/{name}.png')

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd4e185a220_48&reconnect=auto' style='wid…

In [27]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_2landmarks(scene, df_local, mesh_local, ['P4', 'P5'], tangent_axis='z', name='ABG')
    scene.show()
    scene.screenshot(f'output/visual/ABG.png')

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd4e18dd7c0_44&reconnect=auto' style='wid…

In [26]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmark(scene, df_local, mesh_local, 'P6', 'x', name='IG')
    scene.show()
    scene.screenshot(f'output/visual/IG.png')

Widget(value="<iframe src='http://localhost:64774/index.html?ui=P_0x7fd4e18ddd90_43&reconnect=auto' style='wid…